In [ ]:
from __future__ import print_function, division, absolute_import

import GPy
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import ipywidgets as widgets # Widget definitions
from IPython import display # Used to display widgets in the notebook

# Import safeopt from system, alternatively add main folder to path
try:
    import safeopt
except ImportError:
    import sys
    import os
    module_path = os.path.abspath('..')
    sys.path.append(module_path)
    import safeopt

## Define a kernel and function

Here we define a kernel. The function is drawn at random from the GP and is corrupted my Gaussian noise

In [ ]:
noise_var = 0.05 ** 2

# Set fixed Gaussian measurement noise
likelihood = GPy.likelihoods.gaussian.Gaussian(variance=noise_var)
likelihood.constrain_fixed(warning=False);

# Bounds on the inputs variable
bounds = [(-5., 5.), (-5., 5.)]

# set of parameters
parameter_set = safeopt.linearly_spaced_combinations(bounds, 100)

# Define Kernel
kernel = GPy.kern.RBF(input_dim=len(bounds), variance=2., lengthscale=1.0,
                      ARD=True)

# Initial safe point
x0 = np.zeros((1, len(bounds)))

# Generate function with safe initial point at x=0
def sample_safe_fun():
    while True:
        fun = safeopt.sample_gp_function(kernel, bounds, noise_var, 10)
        if fun([0,0], noise=False) > 0.5:
            break
    return fun

## Interactive run of the algorithm

The slow part of running this is the plotting with matplotlib. Consider switching to the 2D level sets.

In [ ]:
# whether to use the swarm optimization
swarm_optimization = True

In [ ]:
button_add = widgets.Button(description='Add sample', tooltip='Sample a new data point for the optimization')
button_reset = widgets.Button(description='Reset', tooltip='Restart the safe optimization algorithm')
button_new = widgets.Button(description='New function', tooltip='Add a new function and reset')
button_3d = widgets.Button(description='Toggle 2D/3D', tooltip='Toggle 2D/3D view')

ms = 20
mew = 7

plot_3d = False
def plot_gp():
    gp_opt.plot(n_samples=50, plot_3d=plot_3d, figure=plt.figure(figsize=(15, 13)))
    
    # Plot last point red
    if gp_opt.gp is not None:
        if plot_3d:
            plt.plot(gp_opt.gp.X[-1, 0, None], gp_opt.gp.X[-1, 1, None], gp_opt.gp.Y[-1, 0, None],
                     'rx', ms=ms, mew=mew, label='Last Point')
        else:
            plt.plot(gp_opt.gp.X[-1, 0], gp_opt.gp.X[-1, 1],
                     'rx', ms=ms, mew=mew, label='Last Point')

    # Ensure we only get one plot
    display.clear_output(wait=True)
    
def toggle_3d(b=None):
    global plot_3d
    plot_3d = not plot_3d
    plot_gp()
button_3d.on_click(toggle_3d)

def new_sample(b=None):
    """Draw a new gp sample"""
    x = gp_opt.optimize()
    gp_opt.add_new_data_point(x, fun(x))
    plot_gp()
button_add.on_click(new_sample)

def reset(b=None):
    """Reset the GP-UCB algorithm"""
    global gp_opt
    gp = GPy.core.GP(x0, fun(x0), kernel, likelihood)
    if swarm_optimization:
        gp_opt = safeopt.SafeOptSwarm([gp], 0., bounds=bounds, threshold=0.2)
    else:
        gp_opt = safeopt.SafeOpt(gp, parameter_set, 0., threshold=0.2)
    plot_gp()
button_reset.on_click(reset)

def new_fun(b=None):
    """Draw a new function from the GP"""
    global fun
    fun = sample_safe_fun()
    reset(b)
button_new.on_click(new_fun)

display.display(button_add, button_new, button_3d, button_reset)
new_fun()